In [22]:
# -*- coding: utf-8 -*-
"""tf25_mnist_ann_noclass.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1W2N7kxyOrofSJ045SjXIlUnyeZ5SXRnk
"""

import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.astype('float32').reshape([-1,784]) / 255.
test_images = test_images.astype('float32').reshape([-1,784]) / 255.
train_labels = tf.one_hot(train_labels, depth=10)
test_labels = tf.one_hot(test_labels, depth=10)
batch_size = 1000
"""
tf.data.Dataset.from_tensor_slices 함수는 tf.data.Dataset 를 생성하는 함수로 
입력된 텐서로부터 slices를 생성합니다. 예를 들어 MNIST의 학습데이터 (60000, 28, 28)가
되면, 60000개의 slices로 만들고 각각의 slice는 28×28의 이미지 크기를 갖게 됩니다.
"""
train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_data = train_data.repeat().shuffle(5000).batch(batch_size)

nH1 = 256
nH2 = 256
nH3 = 256
W_1 = tf.Variable(tf.random.normal([784,nH1]))
b_1 = tf.Variable(tf.random.normal([nH1]))
W_2 = tf.Variable(tf.random.normal([nH1,nH2]))
b_2 = tf.Variable(tf.random.normal([nH2]))
W_3 = tf.Variable(tf.random.normal([nH2,nH3]))
b_3 = tf.Variable(tf.random.normal([nH3]))
W_o = tf.Variable(tf.random.normal([nH3,10]))
b_o = tf.Variable(tf.random.normal([10]))

def model_SoftmaxNN_LC(x):
  H_1 = tf.nn.relu(tf.matmul(x,W_1)+b_1)
  H_2 = tf.nn.relu(tf.matmul(H_1,W_2)+b_2)
  H_3 = tf.nn.relu(tf.matmul(H_2,W_3)+b_3)
  Out = tf.matmul(H_3,W_o)+b_o
  return Out

def cost_Softmax(x,y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=x, labels=y))

def train_optimization(MNIST_images,MNIST_labels):
  with tf.GradientTape() as g:
    model_AI = model_SoftmaxNN_LC(MNIST_images)
    cost = cost_Softmax(model_AI, MNIST_labels)
  gradients = g.gradient(cost, [W_1,W_2,W_3,W_o,b_1,b_2,b_3,b_o])
  tf.optimizers.Adam(0.01).apply_gradients(zip(gradients, [W_1,W_2,W_3,W_o,b_1,b_2,b_3,b_o]))

for epoch in range(10):
  for step, (batch_images, batch_labels) in enumerate(train_data.take(60),1):
    train_optimization(batch_images, batch_labels)
    if step % 10 == 0:
      pred = tf.nn.softmax(model_SoftmaxNN_LC(batch_images))
      accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1),tf.argmax(batch_labels,1)), tf.float32))
      print('Epoch: {} \t Step: {} \t Accuracy: {}'.format(epoch,step,accuracy.numpy()))

print('-'*30)
test_model = tf.nn.softmax(model_SoftmaxNN_LC(test_images))
test_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(test_model,1),tf.argmax(test_labels,1)), tf.float32))
print('Test Accuracy: ', test_accuracy.numpy())

Epoch: 0 	 Step: 10 	 Accuracy: 0.597000002861023
Epoch: 0 	 Step: 20 	 Accuracy: 0.7409999966621399
Epoch: 0 	 Step: 30 	 Accuracy: 0.7889999747276306
Epoch: 0 	 Step: 40 	 Accuracy: 0.8270000219345093
Epoch: 0 	 Step: 50 	 Accuracy: 0.8690000176429749
Epoch: 0 	 Step: 60 	 Accuracy: 0.8980000019073486
Epoch: 1 	 Step: 10 	 Accuracy: 0.8769999742507935
Epoch: 1 	 Step: 20 	 Accuracy: 0.9210000038146973
Epoch: 1 	 Step: 30 	 Accuracy: 0.9259999990463257
Epoch: 1 	 Step: 40 	 Accuracy: 0.9380000233650208
Epoch: 1 	 Step: 50 	 Accuracy: 0.9359999895095825
Epoch: 1 	 Step: 60 	 Accuracy: 0.9419999718666077
Epoch: 2 	 Step: 10 	 Accuracy: 0.9430000185966492
Epoch: 2 	 Step: 20 	 Accuracy: 0.9509999752044678
Epoch: 2 	 Step: 30 	 Accuracy: 0.9330000281333923
Epoch: 2 	 Step: 40 	 Accuracy: 0.9409999847412109
Epoch: 2 	 Step: 50 	 Accuracy: 0.972000002861023
Epoch: 2 	 Step: 60 	 Accuracy: 0.968999981880188
Epoch: 3 	 Step: 10 	 Accuracy: 0.9549999833106995
Epoch: 3 	 Step: 20 	 Accuracy: 0.

In [36]:
#made with PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train_images = datasets.MNIST(root = '../Data', train = True, download = True,
                              transform = transforms.ToTensor())
train_data = datasets.MNIST(root = '../Data', train = True, download = True,
                              transform = transforms.ToTensor())
#라벨 붙여줄 것들
train_labels = torch.zeros(len(train_labels), 10)
test_labels = torch.zeros(len(test_labels), 10)

batch_size = 1000
#train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
#train_data = train_data.repeat().shuffle(5000).batch(batch_size)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset/',train = True,download = True,
                 transform = transforms.Compose([
                     transforms.ToTensor(),
                     transforms.Normalize(mean = (0.5,), std = (0.5,))
                 ])),
    batch_size = batch_size,
    shuffle = True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset',train=False,
                  transform = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.5,), (0,5))
                  ])),
    batch_size = batch_size,
    shuffle = True)


nH1 = 256
nH2 = 256
nH3 = 256
W_1 = torch.randn([784,nH1], requires_grad = True)
W_2 = torch.randn([nH1,nH2], requires_grad = True)
W_3 = torch.randn([nH2,nH3], requires_grad = True)
W_o = torch.randn([nH3,10], requires_grad = True)
b_1 = torch.randn([nH1], requires_grad = True)
b_2 = torch.randn([nH2], requires_grad = True)
b_3 = torch.randn([nH3], requires_grad = True)
b_o = torch.randn([10], requires_grad = True)
optimizer = torch.optim.SGD([W_1, W_2, W_3, W_o, b_1, b_2, b_3, b_o], lr = 0.01)

def model_SoftmaxNN_LC(x):
  H_1 = F.softmax(torch.matmul(x,W_1)+b_1)
  H_2 = F.softmax(torch.matmul(H_1,W_2)+b_2)
  H_3 = F.softmax(torch.matmul(H_2,W_3)+b_3)
  Out = torch.matmul(H_3,W_o)+b_o
  return Out

#def cost_Softmax(x,y):
  #return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=x, labels=y))

def train_optimization(MNIST_images,MNIST_labels):
  with tf.GradientTape() as g:
    model_AI = model_SoftmaxNN_LC(MNIST_images)
    cost = F.cross_entropy(model_AI, MNIST_labels)
  gradients = g.gradient(cost, [W_1,W_2,W_3,W_o,b_1,b_2,b_3,b_o])
  tf.optimizers.Adam(0.01).apply_gradients(zip(gradients, [W_1,W_2,W_3,W_o,b_1,b_2,b_3,b_o]))

In [38]:
for epoch in range(10):
  for step, (batch_images, batch_labels) in enumerate(train_loader.take(60),1):
    train_optimization(batch_images, batch_labels)
    if step % 10 == 0:
      pred = tf.nn.softmax(model_SoftmaxNN_LC(batch_images))
      accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1),tf.argmax(batch_labels,1)), tf.float32))
      print('Epoch: {} \t Step: {} \t Accuracy: {}'.format(epoch,step,accuracy.numpy()))

print('-'*30)
test_model = tf.nn.softmax(model_SoftmaxNN_LC(test_images))
test_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(test_model,1),tf.argmax(test_labels,1)), tf.float32))
print('Test Accuracy: ', test_accuracy.numpy())

AttributeError: ignored